In [1]:
!pip install -U tensorflow_datasets

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfdatasets
tfdatasets.disable_progress_bar()

import numpy as np
import math
import matplotlib.pyplot as plt

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [3]:
dataset, metadata = tfdatasets.load("fashion_mnist", as_supervised=True, with_info=True)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.


In [4]:
# Store class names
class_names = metadata.features["label"].names

In [5]:
# Preprocess the data
train_examples = metadata.splits["train"].num_examples
test_examples = metadata.splits["test"].num_examples

def normalize(a, b):
  a = tf.cast(a, tf.float32)
  a = a/255
  return a, b

train_dataset = train_dataset.map(normalize)
test_dataset = test_dataset.map(normalize)

# Cache data to make training faster
train_dataset = train_dataset.cache()
test_dataset = test_dataset.cache()

In [6]:
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

# Compile the model
model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics = ["accuracy"])

# Train the model
train_dataset = train_dataset.cache().repeat().shuffle(train_examples).batch(32)
test_dataset = test_dataset.cache().batch(32)

model.fit(train_dataset, epochs=5, steps_per_epoch=math.ceil(train_examples/32))

# Evaluate accuracy of the model using the test dataset
test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(test_examples/32))
print("Accuracy on test dataset: ", test_accuracy)

Epoch 1/5
1875/1875 [==============================] - 13s 4ms/step - loss: 0.4960 - accuracy: 0.8259
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3767 - accuracy: 0.8639
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3361 - accuracy: 0.8775
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3098 - accuracy: 0.8860
Epoch 5/5
313/313 [==============================] - 3s 8ms/step - loss: 0.3525 - accuracy: 0.8745
Accuracy on test dataset:  0.8744999766349792


In [7]:
# Now we can make predictions
for test_images, test_labels in test_dataset.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)
predictions.shape

1/1 [==============================] - 0s 80ms/step


(32, 10)

In [9]:
# Making the program open to receiving input from user
index = int(input("Enter the index of the clothing item (between 0 and 31 inclusive) that you want the program to guess: "))
print("The prediction is: "+ class_names[np.argmax(predictions[index])])
print("The correct answer is: "+ class_names[test_labels[index]])
if class_names[np.argmax(predictions[index])] == class_names[test_labels[index]]:
  print("The prediction is correct")
else:
  print("The prediction is wrong")


Enter the index of the clothing item (between 0 and 31 inclusive) that you want the program to guess: 4
The prediction is: Sandal
The correct answer is: Sandal
The prediction is correct
